In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/hse-econ-data-science/andan_2023/main/sem11_ab/cookie_cats.csv')
print(df.shape)
df.head()

(90189, 5)


,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [9]:
df_agg = df.groupby('version')['retention_7'].agg(['mean', 'count'])
df_agg

,mean,count
version,,
gate_30,0.190201,44700
gate_40,0.182000,45489


__а)__  Постройте примерный $95\%$-й доверительный интервал для каждой из двух долей.

$$
\left[\hat p - z_{crit} \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}} ; \quad  \hat p + z_{crit} \sqrt{\frac{\hat p \cdot (1 - \hat p)}{n}} \right]
$$

In [12]:
import scipy.stats as sts
import numpy as np

alpha = 0.05
z_cr = sts.norm().ppf(1 - alpha/2)

df_agg['var'] = df_agg['mean'] * (1 - df_agg['mean'])/df_agg['count']
df_agg['left'] = df_agg['mean'] - z_cr * np.sqrt(df_agg['var'])
df_agg['right'] = df_agg['mean'] + z_cr * np.sqrt(df_agg['var'])
df_agg

,mean,count,var,left,right
version,,,,,
gate_30,0.190201,44700,0.000003,0.186563,0.193840
gate_40,0.182000,45489,0.000003,0.178454,0.185546


__б)__ Постройте примерный $95\%$-й доверительный интервал для разницы долей.

$$
\left[\hat p_A - \hat p_B - z_{crit} \sqrt{\frac{\hat p_A \cdot (1 - \hat p_A)}{n_A} + \frac{\hat p_B \cdot (1 - \hat p_B)}{n_B}} ; \quad  \hat p_A - \hat p_B + z_{crit} \sqrt{\frac{\hat p_A \cdot (1 - \hat p_A)}{n_A} + \frac{\hat p_B \cdot (1 - \hat p_B)}{n_B}} \right]
$$

In [16]:
diff = df_agg['mean']['gate_30'] - df_agg['mean']['gate_40']
diff_sd = np.sqrt(df_agg['var']['gate_30'] +  df_agg['var']['gate_40'])

diff - z_cr*diff_sd, diff + z_cr*diff_sd

(0.0031210442115262808, 0.013281552418885545)

In [18]:
diff/diff_sd, z_cr

(3.1640640401490985, 1.959963984540054)

In [19]:
diff/diff_sd - z_cr

1.2041000556090444

In [20]:
diff

0.008201298315205913

In [22]:
100*(df_agg['mean']['gate_30'] - df_agg['mean']['gate_40'])/df_agg['mean']['gate_30'] 

4.311903489646016

In [26]:
# пусть аб-тест работал неделю и был раскатан неа 5%

df.shape[0] # наверно это чет около 5%

90189

In [27]:
df.shape[0]/0.05 # всего юзеров унашего продукта за неделю

1803780.0

In [28]:
# пусть 20% платят чтобы разлочить ворота (можно это число посчитать по данным)
0.2 * 100 * df.shape[0]/0.05 * diff

295866.75750004244

In [ ]:
# получается что за неделю мы потерям 300 тыщ рублей